# Introduction

The cocoa industry teems with certification schemes each promising to guarantee a fair livelihood for farmers while being sustainably and ethically sourced. In fact, many companies such as Nestle, Barry Callebaut, and Cargill each have their own certification schemes on top of third-party certifications such as Fairtrade, Rain Forest Alliance, and UTZ. This begs the question: just how fair is Fairtrade, et. al?

If Fairtrade is all its cracked up to be, why is there a need for a company-specific sustainability scheme to meet various CSR standards? To this end, Danwatch launched an investigation into Big Cocoa in the Ivory Coast. As part of the research, we put together this map to give us an idea of what is happening on the ground. <b>We wanted to visualize Fairtrade certified producers and suppliers, suspended producers and suppliers, and the overlap (if any) with Big Cocoa.</b>

## Where did we get our information?

For the map below, we worked from two main sources:
* FLOCERT Customer Search [(source)](https://www.flocert.net/about-flocert/customer-search):  FLOCERT is the certification body for Fairtrade and provides a brief record of the name, address or postal box number, and FLOCERT ID for certified and suspended producers (farms) and traders.
* Nestle's supply chain disclosure [(source)](https://www.nestle.com/sites/default/files/asset-library/documents/library/documents/suppliers/supply-chain-disclosure-cocoa-cote-ivoire.pdf): Nestle discloses their Tier 1 as well as Tier 2 suppliers.

I also drew upon other sources such as the [2017 Repertoire des Societes Cooperatives](http://www.agriculture.gouv.ci/uploads/Repertoire_des_soci%C3%A9t%C3%A9s_coop%C3%A9ratives_FILIERES_2017_(2).pdf) and the legal announcements section of [Abidjan.net](https://business.abidjan.net/al/) during the process of trying to locate Fairtrade farms and traders as well as trying to determine where the Nestle farms are based on the farms were referred to from. I will discuss these in detail below. 

## What does the data tell us?

Using the FLOCERT data, we have the names and partial addresses of farms and traders which are certified or suspended as of 21 April 2020. A rough location was logged for each farm or trader based on the partial address provided.

Using the Nestle data, we can see Nestle's named Tier 1 suppliers. Tier 1 suppliers work directly with Nestle. Not surprisingly, there are several members of Big Cocoa, including Barry Callebaut, Cargill, and Blommer/Olam among Nestle's Tier 1 suppliers. Tier 2 suppliers are more interesting: these are direct suppliers to Tier 1 suppliers. As such, the Tier 2 suppliers linked directly to these Big Cocoa players must be the farms where Big Cocoa source their beans.

If we put the data from both datasets together, we can start to answer the question "Where and what are Big Cocoa's activities in the Ivory Coast?" Though the addresses aren't exact, the FLOCERT data is useful in that it provides a glimpse into concentrated areas of farms and traders. Then, we can see if these areas overlap with Nestle's Tier 1 suppliers'activities in the Ivory Coast. Or, we can see if there are matches between the FLOCERT and Nestle farms.

While FLOCERT provides a producer or trader 's name, address, and ID, Nestle only discloses its supplier's name. To flesh out the information from Nestle, I cross-referenced the names with those in the FLOCERT data and the 2017 Repertoire des Societes Cooperatives, which provides a list of Ivoirian cocoa coops, region, activities, and contact details. Since the Repertoire's information is a few years old, I also cross-referenced the legal announcements section of Abidjan.net for signs of the Nestle Tier 2 supplier in question. These legal announcements ranged from the creation of a company to other changes in the company structure. Often, these announcements also included details on the types of activities a coop engages in, such as production, collection, sales, trading, or processing.

## Why make a map?

The data tells us that about 12 of Nestle's Tier 2 suppliers are also Fairtrade certified *or* were Fairtrade certified but have since had their certfications suspended. (I say "about" because we need more information to confirm if one or two of Nestle's Tier 2 suppliers are matches for the list of Fairtrade farms.) This confirms Big Cocoa's activities in the Ivory Coast. However, the data alone does not answer the questions "How fair is Fairtrade?" or "Why do companies need their own certificaton schemes if they already use Fairtrade sources?" or "Why aren't all of a company's suppliers Fairtrade?".

To start to answer these questions and others, Danwatch needed to conduct an on-the-ground investigation. This map allowed us to visualize the spread of the farms we're interested in (farms both Nestle and Fairtrade) and begin to plan our on-the-ground reportage. Check out the rest of this investigation [here]() or keep reading below to see how we cleaned the data and generated our map.

# Working with the Data

Let's start by reading in some necessary libraries, such as `pandas`, `folium`, and `numpy`. Then, let's take a look at the dataset we will be working with.

In [ ]:
# Let's start by importing necessary libraries.
import pandas as pd
import numpy as np
import folium as fo
from folium.plugins import MarkerCluster

# Read in the dataset 'cocoa farms and suppliers.csv'. We'll use the variable cocoa to refer to our dataset.
cocoa = pd.read_csv("cocoa farms and suppliers.csv")

# What kind of data does our dataset contain?
print(cocoa.info())

# Take a peek under the hood. Let's call up a few lines of our dataset.
cocoa.head(5)

## Clean the Data

The dataset is pretty clean, since it was mostly manually compiled. But, there are a few things left to tweak. We don't want to drop any rows, so let's start by filling the null values with 'Not available'. Then, let's make some cosmetic changes to a few of the strings.

In [ ]:
# Fill null values
cocoa = cocoa.fillna('Not available')

# Standardize some names
cocoa['address'] = cocoa['address'].str.replace('Côte d\'Ivoire', 'Ivory Coast')
cocoa['activities'] = cocoa['activities'].str.replace('commercialization', 'trading')

# Capitalization
cocoa['activities'] = cocoa['activities'].str.capitalize()
#cocoa['status'] = cocoa['status'].str.title()
# cocoa['status'] = cocoa['status'][:1].str.capitalize() + cocoa['status'][1:]
# cocoa['status'] = cocoa['status'].str.slice(stop=1).str.upper() + cocoa['status'].str.slice(start=1)

def removeaccents(column_name):
    cocoa[column_name]=cocoa[column_name].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

for col in cocoa.columns:
    if cocoa.dtypes[col] == object:
        removeaccents(col)
        
cocoa.head()

Now that we've cleaned the dataset, let's check what we're working with.

In [ ]:
# Let's check the status, activities, and type columns to make sure there aren't mistakes in the record.

print('status')
print(cocoa['status'].value_counts())
print('activites')
print(cocoa['activities'].value_counts())
print('type')
print(cocoa['type'].value_counts())

The dataset looks better, but let's standardize **Fair Trade** as **Fairtrade** and check the unique values.

In [ ]:
# Let's standardize "Fair Trade" as "Fairtrade"

cocoa['status'] = cocoa['status'].str.replace('Fair Trade', 'Fairtrade')
cocoa['status'].value_counts()

## Get a Google Maps URL
Let's improve the map by adding a Google Maps URL to each point on the map. This allows users to extrapolate from our cocoa map and use the positional data in other investigations.

To get the google URL, use: https://www.google.com/maps/search/?api=1&query=58.698017,-152.522067 format.

Let's add a new column which allows us to easily supply a latitude and longitude for the URL.

In [ ]:
# Now, we're ready to add a new column to store the Google Map url for each record.

def googlemaps(lat,lon):
    return "https://www.google.com/maps/search/?api=1&query={},{}".format(lat,lon)

cocoa['googlemaps'] = np.vectorize(googlemaps)(round(cocoa['lat'], 5), round(cocoa['lon'],5))

cocoa.head(3)

# Create the cocoa map
Now that we've cleaned our data, let's generate the map. We already imported the libraries we needed above. To clean the data, we used the `pandas` library. To generate the map, we'll be using `folium`, which is a wrapper for `Leaflet.js`.

## Setting the basemap

Below we will specify a basemap, the location aroud which to center the map, and the minimum zoom. Since we're focused on the Ivory Coast, I've set `min_zoom` to `7` which prevents users from zooming out too far from the focus of our research. Here's the [documentation](https://python-visualization.github.io/folium/modules.html#folium.folium.Map) for more parameters and details.

In [ ]:
# setting up the map
fo_map = fo.Map(location=[8.022059, -5.684529],
                        zoom_start=7.3,
                        tiles="cartodbpositron",
                    min_zoom=7, control=False
                   )
fo_map.fit_bounds([[10.03,-9.5784571], [5.060599, -1.911002]])

# adding more basemaps
# fo.TileLayer('CartoDB dark_matter', overlay=True, control=False, opacity=.2).add_to(fo_map)
# fo.TileLayer('openstreetmap').add_to(fo_map)

## Creating and customizing markers

There are a few farms which belong to more than one category. For instance, the CAYAT COOP-CA - Cargill (Cooperative Agricole de Yakasse Attobrou) is a Nestle supplier associated with Cargill and is also a certified Fairtrade producer.

I needed to create two icons to represent farms which are Nestle and certified and others which are Nestle and suspended. I created two variable to store the locations of these images, `nc_url` and `ns_url`, which are stored in the same directory as this notebook. Please check the path depending on where you've saved these files.

In [ ]:
# html color codes reusable

certified_c ='#96d876'
nestle_c = '#f2d05d'
suspended_c = '#f69b5d'

# creating featuregroups to toggle

certified_group = fo.FeatureGroup(name='<span style="color:#96d876">&#9673;</span> Certified Fairtrade farms').add_to(fo_map)
suspended_group = fo.FeatureGroup(name='<span style="color:#f69b5d">&#9673;</span> Suspended Fairtrade farms').add_to(fo_map)
nestle_group = fo.FeatureGroup(name='<span style="color:#f2d05d">&#9673;</span> Nestle suppliers').add_to(fo_map)

# customizing clusters
suspended_icons = """
    function(cluster) {
    var childCount = cluster.getChildCount(); 
    var c = ' marker-cluster-';

    if (childCount) {
        c += 'large';
    } 

    return new L.DivIcon({ html: '<div><span>' + childCount + '</span></div>', className: 'marker-cluster' + c, iconSize: new L.Point(40, 40) ,
    spiderfyOnMaxZoom: false, showCoverageOnHover: false, zoomToBoundsOnClick: false});
    }
    """
nestle_icons = """
    function(cluster) {
    var childCount = cluster.getChildCount(); 
    var c = ' marker-cluster-';

    if (childCount) {
        c += 'medium';
    }
    return new L.DivIcon({ html: '<div><span>' + childCount + '</span></div>', className: 'marker-cluster' + c, iconSize: new L.Point(40, 40) ,
    spiderfyOnMaxZoom: false, showCoverageOnHover: false, zoomToBoundsOnClick: false});
    }
    """
certified_icons = """
    function(cluster) {
    var childCount = cluster.getChildCount(); 
    var c = ' marker-cluster-';

    if (childCount < 50) {
        c += 'small';
    }

    return new L.DivIcon({ html: '<div><span>' + childCount + '</span></div>', className: 'marker-cluster' + c, iconSize: new L.Point(40, 40) ,
    spiderfyOnMaxZoom: false, showCoverageOnHover: false, zoomToBoundsOnClick: false});
    }
    """

# creating clusters to aggregate markers
certified_cluster = MarkerCluster(icon_create_function=certified_icons).add_to(certified_group)
suspended_cluster = MarkerCluster(icon_create_function=suspended_icons).add_to(suspended_group)
nestle_cluster = MarkerCluster(icon_create_function=nestle_icons).add_to(nestle_group)

# custom icons
nc_url = r'https://raw.githubusercontent.com/emililiter/Tutorials/aa19fbd826bcf7af3230a94cca2c9ab187b23f6e/cocoa%20map/nestle%20cert.png'
ns_url = r'https://raw.githubusercontent.com/emililiter/Tutorials/aa19fbd826bcf7af3230a94cca2c9ab187b23f6e/cocoa%20map/nestle%20susp.png'

## Isolating the cases which belong to two groups

Above, we created specific markers to accomodate farms which belong to two groups. Now, let's collect these cases into a separate dataframe, `dual_table`.

In [ ]:
# table to collect the "special" cases which belong to two groups
colnames = list(cocoa.columns)
dual_table = pd.DataFrame(columns = colnames)

# loop througn dataframe to mark all the normal points. collect special cases in dual_table above

for i in range(cocoa.shape[0]):
    location = [cocoa['lat'][i], cocoa['lon'][i]]
    popup = ("<b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</b><br><b>{}</b><br>{}<br>".format(
            cocoa['coop'][i], cocoa['status'][i]
        )+"<br><b>Address: </b>{}<br><b>Flocert ID: </b>{}<br><b>Activities: </b>{}<br><br>".format(
        cocoa['address'][i], cocoa['flocert'][i], cocoa['activities'][i])
            +'<a href="' + cocoa['googlemaps'][i] + '"target="_blank">Google Maps <i class="fa fa-external-link" style="font-size:12px"></i></a><br><br>')

    if cocoa['type'][i] == 'suspended':
        fo.CircleMarker(location = location,
                    fill=True,
                    popup= popup, 
                    color = suspended_c
                   ).add_to(suspended_cluster)
        
    elif cocoa['type'][i] == 'certified':
        fo.CircleMarker(location = location,
                    fill=True,
                    popup= popup, 
                    color = certified_c
                   ).add_to(certified_cluster)
    elif cocoa['type'][i] == 'nestle':
        fo.CircleMarker(location = location,
                    fill=True,
                    popup= popup, 
                    color= nestle_c
                   ).add_to(nestle_cluster)    
        
    elif cocoa['type'][i] == 'dual':
        dual_table = dual_table.append(cocoa.iloc[i],ignore_index=True)

# Let's see the special cases below.

print(dual_table.info())
dual_table.head()

## Categorize the special cases

Now that we've isolated the cases which belong to two different categories, we can add them to either the certified or suspended categories. 

In [ ]:
# Now for the special cases which are nestle and also something else

leftovers=pd.DataFrame(columns=colnames)

# Loop through the special cases, adding them to cert or susp layers

for i in range (dual_table.shape[0]):
    location = [dual_table['lat'][i], dual_table['lon'][i]]
    popup = ("<b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</b><br><b>{}</b><br>{}<br>".format(
            dual_table['coop'][i], dual_table['status'][i]
        )+"<br><b>Address: </b>{}<br><b>Flocert ID: </b>{}<br><b>Activities: </b>{}<br><br>".format(
        dual_table['address'][i], dual_table['flocert'][i], dual_table['activities'][i])
            +'<a href="' + dual_table['googlemaps'][i] + '"target="_blank">Google Maps <i class="fa fa-external-link" style="font-size:12px"></i> </a><br><br>')
    
    if 'certified' in dual_table['status'][i]:
        fo.Marker(location = location,
                    fill=True, 
 #                   tooltip = tooltip,
                    popup = popup,
                    icon = fo.CustomIcon(icon_image = nc_url,icon_size=(25,25))
                   ).add_to(certified_cluster)
        
    elif 'suspended' in dual_table['status'][i]:
        fo.Marker(location = location,
                    fill=True, 
#                    tooltip = tooltip,
                    popup = popup,
                    icon=fo.CustomIcon(icon_image=ns_url,icon_size=(25,25))
                   ).add_to(suspended_cluster)
    else:
        leftovers = leftovers.append(dual_table.iloc[i],ignore_index=True)
        
# Loop througn again to add to nestle layer
        
for i in range (dual_table.shape[0]):
    location = [dual_table['lat'][i], dual_table['lon'][i]]
    popup = ("<b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</b><br><b>{}</b><br>{}<br>".format(
            dual_table['coop'][i], dual_table['status'][i]
        )+"<br><b>Address: </b>{}<br><b>Flocert ID: </b>{}<br><b>Activities: </b>{}<br><br>".format(
        dual_table['address'][i], dual_table['flocert'][i], dual_table['activities'][i])
            +'<a href="' + dual_table['googlemaps'][i] + '"target="_blank">Google Maps <i class="fa fa-external-link" style="font-size:12px"></i> </a><br><br>')
    
    if 'certified' in dual_table['status'][i]:
        fo.Marker(location = location,
                    fill=True,
                  popup= popup, 
#                    tooltip = tooltip,
                    icon=fo.CustomIcon(icon_image=nc_url,icon_size=(25,25))
                   ).add_to(nestle_cluster)
        
        
    elif 'suspended' in dual_table['status'][i]:
        fo.Marker(location = location,
                    fill=True,
                  popup= popup, 
#                    tooltip = tooltip,
                    icon = fo.CustomIcon(icon_image=ns_url,icon_size=(25,25))
                   ).add_to(nestle_cluster)

As we looped through the cases above, we added cases to one of two categories and assigned the leftovers to a dataframe, `leftovers`. Let's check there are no more cases left in the `leftovers` dataframe.

In [ ]:
# Let's check there's nothing left in the leftovers dataframe.
leftovers.info

## Generate the map

Let's generate the cocoa map with the methods below.

In [ ]:
# Time to generate the map
fo.LayerControl().add_to(fo_map)
fo_map

## Save map as HTML

Looks good! Let's use the following method to save the map as HTML.

In [ ]:
fo_map.save('cocoa_map.html')